In [2]:
import requests

In [7]:
BACKEND_HOST_IP="10.74.4.25"
KONG_URL="http://163.172.181.83:443/config"
BACKEND_URI="http://10.74.4.25:5000/"
FRONTEND_URI="http://10.74.4.25:4000/"

In [8]:
services = [
    {
        'name': "enki_api",
        'url': BACKEND_URI,
        'path': '/api/enki',
        "auth":"oidc"
    },
        {
        'name': "enki_front",
        'url': FRONTEND_URI,
        'path': '/enki',
        "auth":"oidc"
    },
    {
        'name': "enki_echange",
        'url': FRONTEND_URI,
        'path': '/enki',
        "auth":"api_key"
    }
]

In [17]:
response = requests.get(f'{KONG_URL}/routes')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'{KONG_URL}/routes/{_id}')
response = requests.get(f'{KONG_URL}/services')
for _id in [e["id"] for e in response.json()["data"]]:
    requests.delete(f'{KONG_URL}/services/{_id}')

In [18]:
apikey = "my-api-key"

In [19]:
for service in services : 
    response = requests.post(f'{KONG_URL}/services', data={
        'name': service["name"],
        'url': service["url"]
    })
    created_service_id = response.json()["id"]
    response = requests.post(f'{KONG_URL}/services/{service["name"]}/routes', data={
        'service.id': created_service_id,
        'paths[]':service["path"],
    })
    route_id = response.json()["id"]
    if service["auth"] == "api_key":
        response = requests.post(f'{KONG_URL}/routes/{route_id}/plugins', data={
            'name': "key-auth",
            'config.key_names': apikey,
        })
    elif service["auth"] == "oidc":
        pass

In [20]:
response.json()

{'config': {'anonymous': None,
  'hide_credentials': False,
  'key_in_body': False,
  'key_in_header': True,
  'key_in_query': True,
  'key_names': ['my-api-key'],
  'run_on_preflight': True},
 'consumer': None,
 'created_at': 1618586278,
 'enabled': True,
 'id': 'dfe95a15-c221-4a31-9e14-b6febd2b35f8',
 'name': 'key-auth',
 'protocols': ['grpc', 'grpcs', 'http', 'https'],
 'route': {'id': '4bdb34e5-da50-4cb3-a0a7-41a9ec481962'},
 'service': None,
 'tags': None}